In [33]:
import pymysql

# Define the MySQL connection function
def get_connection():
    return pymysql.connect(
        host="mysql.clarksonmsda.org",
        user="singarv",
        password="Clarkson23@#",
        database="singarv_accident",
        autocommit=True,
        cursorclass=pymysql.cursors.DictCursor
    )

In [35]:
def create_tables():
    connection = get_connection()
    with connection.cursor() as cursor:
        # Create Accidents table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Accidents (
            Accident_Index VARCHAR(50) PRIMARY KEY,
            Longitude DOUBLE,
            Latitude DOUBLE,
            Accident_Severity INT,
            Date DATE,
            Weather_Conditions VARCHAR(255),
            Light_Conditions VARCHAR(255),
            Road_Surface_Conditions VARCHAR(255),
            Number_of_Vehicles INT,
            Number_of_Casualties INT
        );
        """)
        # Create Vehicles table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Vehicles (
            Accident_Index VARCHAR(50),
            Vehicle_Reference INT,
            Vehicle_Type VARCHAR(255),
            Vehicle_Manoeuvre VARCHAR(255),
            Skidding_and_Overturning VARCHAR(255),
            FOREIGN KEY (Accident_Index) REFERENCES Accidents(Accident_Index)
        );
        """)
        # Create Casualties table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Casualties (
            Accident_Index VARCHAR(50),
            Casualty_Reference INT,
            Casualty_Severity INT,
            Casualty_Class VARCHAR(255),
            Age_of_Casualty INT,
            Sex_of_Casualty VARCHAR(50),
            FOREIGN KEY (Accident_Index) REFERENCES Accidents(Accident_Index)
        );
        """)
    connection.close()
    print("Tables created successfully.")

create_tables()

Tables created successfully.


In [36]:
import pandas as pd

# Clean column names for MySQL compatibility
def clean_column_names(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("[()]", "", regex=True)
        .str.replace("^\\d+", "col_", regex=True)
    )
    return df

# Filter columns to match the database schema
def filter_columns(df, required_columns):
    return df[required_columns]

# Handle missing values for each table
def handle_missing_values(df, table_name):
    if table_name == 'Accidents':
        df = df.fillna({
            'Longitude': 0.0,
            'Latitude': 0.0,
            'Weather_Conditions': 'Unknown',
            'Light_Conditions': 'Unknown',
            'Road_Surface_Conditions': 'Unknown',
            'Number_of_Vehicles': 0,
            'Number_of_Casualties': 0,
            'Date': '1970-01-01'
        })
    elif table_name == 'Vehicles':
        df = df.fillna({
            'Vehicle_Type': 'Unknown',
            'Vehicle_Manoeuvre': 'Unknown',
            'Skidding_and_Overturning': 'Unknown'
        })
    elif table_name == 'Casualties':
        df = df.fillna({
            'Casualty_Severity': 3,
            'Casualty_Class': 'Unknown',
            'Age_of_Casualty': -1,
            'Sex_of_Casualty': 'Unknown'
        })
    return df

# Load datasets
accidents_df = pd.read_csv('Accidents.csv')
vehicles_df = pd.read_csv('Vehicles.csv')
casualties_df = pd.read_csv('Casualties.csv')

# Clean column names
accidents_df = clean_column_names(accidents_df)
vehicles_df = clean_column_names(vehicles_df)
casualties_df = clean_column_names(casualties_df)

# Convert Date column to YYYY-MM-DD format
if 'Date' in accidents_df.columns:
    accidents_df['Date'] = pd.to_datetime(accidents_df['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Relevant columns for each table
accident_columns = [
    'Accident_Index', 'Longitude', 'Latitude', 'Accident_Severity', 'Date',
    'Weather_Conditions', 'Light_Conditions', 'Road_Surface_Conditions',
    'Number_of_Vehicles', 'Number_of_Casualties'
]
vehicle_columns = [
    'Accident_Index', 'Vehicle_Reference', 'Vehicle_Type',
    'Vehicle_Manoeuvre', 'Skidding_and_Overturning'
]
casualty_columns = [
    'Accident_Index', 'Casualty_Reference', 'Casualty_Severity',
    'Casualty_Class', 'Age_of_Casualty', 'Sex_of_Casualty'
]

# Filter datasets
accidents_df = filter_columns(accidents_df, accident_columns)
vehicles_df = filter_columns(vehicles_df, vehicle_columns)
casualties_df = filter_columns(casualties_df, casualty_columns)

# Handle missing values
accidents_df = handle_missing_values(accidents_df, 'Accidents')
vehicles_df = handle_missing_values(vehicles_df, 'Vehicles')
casualties_df = handle_missing_values(casualties_df, 'Casualties')

C:\Users\onevs\AppData\Local\Temp\ipykernel_7788\348827966.py:48: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents_df = pd.read_csv('Accidents.csv')


In [39]:
def insert_data_batch(df, table_name, batch_size=1000):
    connection = get_connection()
    with connection.cursor() as cursor:
        for start in range(0, len(df), batch_size):
            batch = df.iloc[start:start + batch_size]
            cols = ', '.join(batch.columns)  # Extract column names
            vals = ', '.join(['%s'] * len(batch.columns))  # Prepare placeholders
            query = f"INSERT INTO {table_name} ({cols}) VALUES ({vals})"
            data = [tuple(row) for row in batch.to_numpy()]  # Prepare data
            try:
                cursor.executemany(query, data)  # Execute batch insertion
                print(f"Inserted {start + len(batch)} rows into {table_name}")
            except Exception as e:
                print(f"Error inserting batch starting at row {start}: {e}")
    connection.close()

# Insert data into tables
print("Inserting into Accidents table...")
insert_data_batch(accidents_df, 'Accidents')

print("Inserting into Vehicles table...")
insert_data_batch(vehicles_df, 'Vehicles')

print("Inserting into Casualties table...")
insert_data_batch(casualties_df, 'Casualties') 

print("All data inserted successfully!")

Inserting into Vehicles table...
Inserted 1000 rows into Vehicles
Inserted 2000 rows into Vehicles
Inserted 3000 rows into Vehicles
Inserted 4000 rows into Vehicles
Inserted 5000 rows into Vehicles
Inserted 6000 rows into Vehicles
Inserted 7000 rows into Vehicles
Inserted 8000 rows into Vehicles
Inserted 9000 rows into Vehicles
Inserted 10000 rows into Vehicles
Inserted 11000 rows into Vehicles
Inserted 12000 rows into Vehicles
Inserted 13000 rows into Vehicles
Inserted 14000 rows into Vehicles
Inserted 15000 rows into Vehicles
Inserted 16000 rows into Vehicles
Inserted 17000 rows into Vehicles
Inserted 18000 rows into Vehicles
Inserted 19000 rows into Vehicles
Inserted 20000 rows into Vehicles
Inserted 21000 rows into Vehicles
Inserted 22000 rows into Vehicles
Inserted 23000 rows into Vehicles
Inserted 24000 rows into Vehicles
Inserted 25000 rows into Vehicles
Inserted 26000 rows into Vehicles
Inserted 27000 rows into Vehicles
Inserted 28000 rows into Vehicles
Inserted 29000 rows into